# Start script

## Import librares

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
# ee.Initialize(project='gee-antonbiatov')
ee.Initialize(project='MY-GEE-PROJECT')


In [3]:
import subprocess
import random
import time

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

## Input variables

In [4]:
YEARS = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

STARTDAY=1
ENDDAY=250
# INTERVAL = ENDDAY - STARTDAY
scale = 15

my_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']

CLOUD_FILTER = 50



In [5]:
geom = ee.Geometry.Polygon(
        [
            [
             [50.230865478515625,26.309419586402797],[50.22674560546875,26.30934264658286],[50.2239990234375,26.309419586402797],[50.21721839904785,26.308650185904614],
             [50.21498680114746,26.308727126184305],[50.21309852600098,26.309419586402797],[50.17979621887207,26.3150360552725],[50.177650451660156,26.310881433275505],
             [50.175161361694336,26.3072652521349],[50.17250061035156,26.303418127049035],[50.16975402832031,26.301032845368663],[50.165462493896484,26.297878042601646],
             [50.1719856262207,26.292953300840438],[50.1774787902832,26.28972132539388],[50.18202781677246,26.288413119250144],[50.19155502319336,26.285565796050843],
             [50.19850730895996,26.28356493253137],[50.204429626464844,26.281794909113252],[50.20820617675781,26.280717490333167],[50.212240219116204,26.28002485869133],
             [50.22425651550293,26.27833174171643],[50.230865478515625,26.309419586402797]
            ]
        ]
)
feature = ee.Feature(geom, {})

AOI = feature.geometry()

## Create functions

In [6]:
def maskCloud1(image):
    # Add a cloud score band.  It is automatically called 'cloud'.
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    # Create a mask from the cloud score and combine it with the image mask.
    mask = scored.select(['cloud']).lte(20);
    # Apply the mask to the image and display the result.
    return image.updateMask(mask)

def select_my_bands(img):
    return img.select(my_bands)


def img_panshapining(image):
    rgb = image.select('B4', 'B5', 'B3').clip(AOI)
    pan = image.select('B8').clip(AOI)
    # Convert to HSV, swap in the pan band, and convert back to RGB.
    huesat = rgb.rgbToHsv().select('hue', 'saturation');
    upres = ee.Image.cat(huesat, pan).hsvToRgb();
    return upres

def img_panshapining764(image):
    rgb = image.select('B7', 'B6', 'B4').clip(AOI)
    pan = image.select('B8').clip(AOI)
    # Convert to HSV, swap in the pan band, and convert back to RGB.
    huesat = rgb.rgbToHsv().select('hue', 'saturation');
    upres = ee.Image.cat(huesat, pan).hsvToRgb();
    return upres



In [7]:
def get_l8_col(aoi, STARTYEAR, ENDYEAR,STARTDAY,ENDDAY):
    l8_col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
        .filterBounds(AOI)
        .filter(ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'))
        .filter(ee.Filter.dayOfYear(STARTDAY, ENDDAY))
        .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER))
        .map(maskCloud1)
        .map(select_my_bands)
    )
    return ee.ImageCollection(l8_col)

In [8]:
def get_l8_upres_col(aoi, STARTYEAR, ENDYEAR,STARTDAY,ENDDAY):
    l8_col_upres = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
        .filterBounds(AOI)
        .filter(ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'))
        .filter(ee.Filter.dayOfYear(STARTDAY, ENDDAY))
        .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER))
        .map(maskCloud1)
        .map(img_panshapining)
    )
    return ee.ImageCollection(l8_col_upres)

In [9]:
def get_l8_764_upres_col(aoi, STARTYEAR, ENDYEAR,STARTDAY,ENDDAY):
    l8_764_col_upres = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
        .filterBounds(AOI)
        .filter(ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'))
        .filter(ee.Filter.dayOfYear(STARTDAY, ENDDAY))
        .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER))
        .map(maskCloud1)
        .map(img_panshapining764)
    )
    return ee.ImageCollection(l8_764_col_upres)

In [10]:
def median_image_generator(YEAR):
    l8_col = get_l8_col(AOI, YEAR, YEAR, STARTDAY,ENDDAY)
    med=l8_col.median().clip(AOI).set('year',YEAR).set('col_size', l8_col.size().getInfo())
    return med

In [11]:
def median_image_upres_generator(YEAR):
    l8_upres_col = get_l8_upres_col(AOI, YEAR, YEAR, STARTDAY,ENDDAY)
    med=l8_upres_col.median().clip(AOI).set('year',YEAR).set('col_size', l8_upres_col.size().getInfo())
    return med

In [12]:
def median_image_764_upres_generator(YEAR):
    l8_764_upres_col = get_l8_764_upres_col(AOI, YEAR, YEAR, STARTDAY,ENDDAY)
    med=l8_764_upres_col.median().clip(AOI).set('year',YEAR).set('col_size', l8_764_upres_col.size().getInfo())
    return med

## Check years with data

In [13]:
for YEAR in YEARS:
    img = median_image_generator(YEAR)
    print(str(YEAR)+' Image collection size: '+str(img.get('col_size').getInfo()))

2013 Image collection size: 19
2014 Image collection size: 28
2015 Image collection size: 29
2016 Image collection size: 26
2017 Image collection size: 26
2018 Image collection size: 27
2019 Image collection size: 24
2020 Image collection size: 26
2021 Image collection size: 31
2022 Image collection size: 28


# Visualization on webmap

## 30m

In [14]:
rgbVis_False_color_453 = {'min': 0, 'max': 0.5, 'bands': ['B4', 'B5', 'B3'],};
rgbVis_False_color_764 = {'min': 0, 'max': 0.5, 'bands': ['B7', 'B6', 'B4'],};

Map = []
center = AOI.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=14)

Map.addLayer(AOI, {}, 'AOI', True, 1);

YEARS_to_export = []

for YEAR in YEARS:
    img = median_image_generator(YEAR)

    if img.get('col_size').getInfo() >= 1 :
        Map.addLayer(img, rgbVis_False_color_453, 'med_'+str(YEAR)+'_453', True);
        Map.addLayer(img, rgbVis_False_color_764, 'med_'+str(YEAR)+'_764', True);
        print(str(YEAR)+' Image collection size: '+str(img.get('col_size').getInfo()))
        YEARS_to_export.append(YEAR)

print('YEARS_to_export: ', YEARS_to_export)

Map

2013 Image collection size: 19
2014 Image collection size: 28
2015 Image collection size: 29
2016 Image collection size: 26
2017 Image collection size: 26
2018 Image collection size: 27
2019 Image collection size: 24
2020 Image collection size: 26
2021 Image collection size: 31
2022 Image collection size: 28
YEARS_to_export:  [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


Map(center=[26.297459878673237, 50.20115646416002], controls=(WidgetControl(options=['position', 'transparent_…

## 15m

In [17]:
rgbVis_False_color_RGB = {'min': 0, 'max': 0.4, 'bands': ['red', 'green', 'blue'],};

Map = []
center = AOI.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=14)

Map.addLayer(AOI, {}, 'AOI', True, 1);

for YEAR in YEARS:
    img_upres = median_image_upres_generator(YEAR)
    img_upres_764 = median_image_764_upres_generator(YEAR)
    if img_upres.get('col_size').getInfo() >= 1 :
        Map.addLayer(img_upres, rgbVis_False_color_RGB, 'med_'+str(YEAR)+'_upres_453', True);
        Map.addLayer(img_upres_764, rgbVis_False_color_RGB, 'med_'+str(YEAR)+'_upres_764', True);
        print(str(YEAR)+' Image collection size: '+str(img.get('col_size').getInfo()))


Map

2013 Image collection size: 28
2014 Image collection size: 28
2015 Image collection size: 28
2016 Image collection size: 28
2017 Image collection size: 28
2018 Image collection size: 28
2019 Image collection size: 28
2020 Image collection size: 28
2021 Image collection size: 28
2022 Image collection size: 28


Map(center=[26.297459878670836, 50.20115646415958], controls=(WidgetControl(options=['position', 'transparent_…

## Comparison 30m VS 15m resolution

In [18]:
left_img = median_image_generator(2021)
right_img = median_image_upres_generator(2021)

left_layer = geemap.ee_tile_layer(left_img, rgbVis_False_color_453, 'res_30m_2021')
right_layer = geemap.ee_tile_layer(right_img, rgbVis_False_color_RGB, 'res_15m_2021')

In [19]:
Map = []
center = AOI.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=15)

Map.split_map(left_layer, right_layer)
Map

Map(center=[26.297459878673237, 50.20115646416002], controls=(ZoomControl(options=['position', 'zoom_in_text',…

# Export data

In [ ]:
for YEAR in YEARS_to_export:
    img = median_image_upres_generator(YEAR)
    ee.batch.Export.image.toDrive(image = img, region=AOI, folder="GEE_data_export",
    description = 'Khobar_L8_pansharp_'+str(YEAR), crs='EPSG:4326', scale = scale, maxPixels = 1e10,fileFormat = 'GeoTIFF').start()
    time.sleep(round(random.uniform(15, 35)))


In [ ]:
for YEAR in YEARS_to_export:
    img = median_image_764_upres_generator(YEAR)
    ee.batch.Export.image.toDrive(image = img, region=AOI, folder="GEE_data_export",
    description = 'Khobar_L8_pansharp_'+str(YEAR)+'_FalseColorUrban', crs='EPSG:4326', scale = scale, maxPixels = 1e10,fileFormat = 'GeoTIFF').start()
    time.sleep(round(random.uniform(15, 35)))

In [ ]:
for YEAR in YEARS_to_export:
    img = median_image_generator(YEAR)
    ee.batch.Export.image.toDrive(image = img.select(['B4', 'B5', 'B3']), region=AOI, folder="GEE_data_export",
    description = 'Khobar_L8_'+str(YEAR), crs='EPSG:4326', scale = scale, maxPixels = 1e10,fileFormat = 'GeoTIFF').start()
    time.sleep(round(random.uniform(15, 35)))